In [20]:
# Install Pytorch
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
# Import for the neccesaary classes
import numpy as np
import torch
import torchvision # provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn

In [22]:
# set the MNIST trainSet
train_set = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform = transforms.Compose([
          transforms.ToTensor()
    ])
)
print(len(train_set))

sample = next(iter(train_set))

60000


Create the Neural Network which takes two inputs i,e MNIST image and a random No and produce two outputs i,e number printed on the image and the Sum of Number on the image and   randon Number

In [23]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=10)
  
  def forward(self, t,t1):
    # input layer
    x = t
    # conv1 layer
    x = self.conv1(x) # 28 > 24
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 28 | 24 | 12
    # conv2 layer
    x = self.conv2(x) # 12
    x = F.relu(x) # 8
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 12 | 8 | 4 >> 12x4x4
    # reshapre
    x = x.reshape(-1, 12 * 4 * 4)
    # fc1 layer
    x = self.fc1(x)
    x = F.relu(x)
    # fc2 layer
    x = self.fc2(x)
    x = F.relu(x)
    # output layer
    x = self.out(x)
    # Note Adding two Numbers i,e below is Sum of (Random no Passed and digit on MNIST image passed)
    t1 = F.softmax(x, dim=1).argmax(dim=1) + t1
    return x,t1

Check whether the Network Class is working as expected. In our case it should except two input Parameters  and return two outputs


In [24]:
sample = next(iter(train_set)) 
image, label = sample
print('Label = '+ str(label))
network_test = Network()
pred = network_test(image,6)
print(pred[0])
print(pred[1])

Label = 5
tensor([[ 0.0921, -0.0102, -0.0825, -0.0105, -0.0536, -0.0646,  0.0149, -0.1035,
         -0.0100, -0.0975]], grad_fn=<AddmmBackward0>)
tensor([6])


Define Helper Funtions

In [25]:
import tensorflow as tf
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

def get_num_correct_Sum(preds, labels):
  return preds.eq(labels).sum().item()

def get_mean_absolute_loss(a,b):
  c = torch.abs(a - b) 
  return c.mean(dtype=torch.float32).item()  


Train the Model

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
network = Network().to(device)

batchSize = 100
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batchSize)
optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(10):
  total_loss = 0
  total_correct = 0
  total_loss_sum = 0
  total_correct_sum = 0
  for batch in train_loader: # Get Batch
      images, labels = batch 
      # Random No generation Strategy . 
      # Note -: Size should be same as above batch size here its 100
      random_tensor = torch.randint(low=0, high=9 , size=(batchSize,))
      #labels_Sum_Image_RandonNo is the sum of Random No and sample image 
      labels_Sum_Image_RandonNo = labels + random_tensor
      #print('Labels Sum = '+ str(labels_Sum_Image_RandonNo))
      preds = network(images,random_tensor) # Pass Batch
      loss = F.cross_entropy(preds[0], labels) # Calculate Loss
      #print('Predicted Sum = '+ str(preds[1]))
      #loss = mse_loss(preds[0].numpy(), labels.numpy()) # Calculate Loss
      optimizer.zero_grad()
      loss.backward() # Calculate Gradients
      optimizer.step() # Update Weights

      total_loss += loss.item()
      total_correct += get_num_correct(preds[0], labels)
      
      # Loss function used is Mean absolute Error
      total_loss_sum += get_mean_absolute_loss(preds[1], labels_Sum_Image_RandonNo)
      total_correct_sum += get_num_correct_Sum(preds[1], labels_Sum_Image_RandonNo)

  print(
      "epoch:", epoch, 
      "total_correct:", total_correct, 
      "loss:", total_loss,
      "total_correct_sum:", total_correct_sum,
      "total_loss_sum:", total_loss_sum
  )

epoch: 0 total_correct: 55936 loss: 126.80575991398655 total_correct_sum: 55936 total_loss_sum: 148.69000012427568
epoch: 1 total_correct: 58420 loss: 53.99028688244289 total_correct_sum: 58420 total_loss_sum: 57.06000006198883
epoch: 2 total_correct: 58656 loss: 46.707657108374406 total_correct_sum: 58656 total_loss_sum: 48.32999990694225
epoch: 3 total_correct: 58958 loss: 35.754497959365835 total_correct_sum: 58958 total_loss_sum: 37.72999989800155
epoch: 4 total_correct: 58954 loss: 38.11940154446347 total_correct_sum: 58954 total_loss_sum: 36.35999991931021
epoch: 5 total_correct: 58911 loss: 39.816984436864004 total_correct_sum: 58911 total_loss_sum: 40.989999959245324
epoch: 6 total_correct: 59081 loss: 34.73771578959713 total_correct_sum: 59081 total_loss_sum: 33.579999931156635
epoch: 7 total_correct: 58988 loss: 40.312292085553054 total_correct_sum: 58988 total_loss_sum: 36.52999991364777
epoch: 8 total_correct: 59095 loss: 34.25282109384716 total_correct_sum: 59095 total_los